In [1]:
import os

os.chdir("../")

In [2]:
!pwd

/Users/jakubszumski/GitRepos/en-de-backtranslation


In [2]:
import wandb
wandb.login()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /Users/jakubszumski/.netrc


True

In [3]:
!fairseq-preprocess --source-lang en --target-lang de \
  --trainpref ./data/datasets/unaugmented/original.train \
  --validpref ./data/datasets/unaugmented/original.val  \
  --testpref  ./data/datasets/unaugmented/original.test  \
  --destdir ./data/binarized/original.tokenized.en-de \
  --thresholdsrc 2 \
  --thresholdtgt 2

zsh:1: command not found: fairseq-preprocess


In [ ]:
! fairseq-train data-bin/multi30k.tokenized.de-en \
  --arch transformer \
  --dropout 0.1 \
  --attention-dropout 0.1 \
  --activation-dropout 0.1 \
  --encoder-embed-dim 256 \
  --encoder-ffn-embed-dim 512 \
  --encoder-layers 3 \
  --encoder-attention-heads 8 \
  --encoder-learned-pos \
  --decoder-embed-dim 256 \
  --decoder-ffn-embed-dim 512 \
  --decoder-layers 3 \
  --decoder-attention-heads 8 \
  --decoder-learned-pos \
  --max-epoch 10 \
  --optimizer adam \
  --lr 5e-4 \
  --batch-size 128 \
  --seed 1 \
  --wandb-project "Multi 30K En to De Translation"

In [ ]:
# ckpt_best, beam=5
! fairseq-generate data-bin/multi30k.tokenized.de-en \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 128 \
    --beam 5 \
    --seed 1 \
    --scoring bleu \
    --wandb-project "Multi 30K En to De Translation"

In [ ]:
# ckpt_best, beam=10
! fairseq-generate data-bin/multi30k.tokenized.de-en \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 128 \
    --beam 10 \
    # 1, 3, 5, 10, 50
    --seed 1 \
    --scoring bleu \
    --quiet \
    --wandb-project "Multi 30K En to De Translation"

2022-05-11 19:06:49 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-11 19:06:51 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': 'Multi 30K En to De Translation', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profil

# Generate Average Checkpoint (Ensemble)

In [ ]:
! python fairseq/scripts/average_checkpoints.py --inputs checkpoints --num-epoch-checkpoints 5 --output checkpoints/averaged.pt

In [ ]:
# ckpt_avg, beam=5
! fairseq-generate data-bin/multi30k.tokenized.de-en \
    --path checkpoints/averaged.pt \
    --batch-size 128 \
    --beam 5 \
    --seed 1 \
    --scoring bleu \
    --quiet \
    --wandb-project "Multi 30K En to De Translation"

In [ ]:
# ckpt_avg, beam=10
! fairseq-generate data-bin/multi30k.tokenized.de-en \
    --path checkpoints/averaged.pt \
    --batch-size 128 \
    --beam 10 \
    --seed 1 \
    --scoring bleu \
    --quiet \
    --wandb-project "Multi 30K En to De Translation"

# Export the model

In [ ]:
! mkdir -p trained_model
! cp data-bin/multi30k.tokenized.de-en/dict.de.txt trained_model/dict.de.txt
! cp data-bin/multi30k.tokenized.de-en/dict.en.txt trained_model/dict.en.txt
! cp checkpoints/averaged.pt trained_model/model.pt

In [ ]:
! fairseq-interactive \
  --path trained_model/model.pt \
  --source-lang de --target-lang en \
  --tokenizer moses \
  --task translation --cpu \
  --beam 5 \
  trained_model/